In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

In [47]:
structure = [['Correct Data Structure'],
             ['Vocabulary'],
             ['char ---> ind AND ind ---> char'], 
             ['Use indexes to ENCODE the text'],
             ['TensorSlicesDataset'],
             ['Sequences'],
             ['Shuffle & Batch'],
             ['Model'],
             ]

In [9]:
path_to_file = "shakespeare.txt"

In [10]:
text = open(path_to_file, 'r').read()

In [18]:
print(text[150:1000])

ecease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su


In [19]:
vocab = sorted(set(text))

In [21]:
len(vocab) #It is important when designing the final Dense layer

84

In [22]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [28]:
char_to_ind['I']

34

In [29]:
ind_to_char =np.array(vocab)

In [40]:
ind_to_char[34]

'I'

In [46]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [49]:
encoded_text.shape

(5445609,)

In [58]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [57]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

In [61]:
seq_length = 130

In [62]:
total_num_seq = len(text) // (seq_length+1)

In [64]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [65]:
for item in char_dataset.take(500):
    print(item.numpy())

0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
12
0
1
1
31
73
70
68
1
61
56
64
73
60
74
75
1
58
73
60
56
75
76
73
60
74
1
78
60
1
59
60
74
64
73
60
1
64
69
58
73
60
56
74
60
8
0
1
1
45
63
56
75
1
75
63
60
73
60
57
80
1
57
60
56
76
75
80
5
74
1
73
70
74
60
1
68
64
62
63
75
1
69
60
77
60
73
1
59
64
60
8
0
1
1
27
76
75
1
56
74
1
75
63
60
1
73
64
71
60
73
1
74
63
70
76
67
59
1
57
80
1
75
64
68
60
1
59
60
58
60
56
74
60
8
0
1
1
33
64
74
1
75
60
69
59
60
73
1
63
60
64
73
1
68
64
62
63
75
1
57
60
56
73
1
63
64
74
1
68
60
68
70
73
80
21
0
1
1
27
76
75
1
75
63
70
76
1
58
70
69
75
73
56
58
75
60
59
1
75
70
1
75
63
64
69
60
1
70
78
69
1
57
73
64
62
63
75
1
60
80
60
74
8
0
1
1
31
60
60
59
5
74
75
1
75
63
80
1
67
64
62
63
75
5
74
1
61
67
56
68
60
1
78
64
75
63
1
74
60
67
61
9
74
76
57
74
75
56
69
75
64
56
67
1
61
76
60
67
8
0
1
1
38
56
66
64
69
62
1
56
1
61
56
68
64
69
60
1
78
63
60
73
60
1
56
57
76
69
59
56
69
58
60
1
67
64
60
74
8
0
1
1
45
63
80
1
74
60
67
61
1
75
63
80
1
61
70
60
8
1
75
70
1
75
63


In [67]:
for item in char_dataset.take(500):
    print(ind_to_char[item.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [68]:
sequences = char_dataset.batch(seq_length+1,drop_remainder=True)

In [69]:
type(sequences)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [71]:
def create_sequence_targets(seq):
    input_txt = seq[:-1] # Hello my nam
    target_txt = seq[1:] # ello my name
    return input_txt,target_txt

In [72]:
dataset = sequences.map(create_sequence_targets)

In [75]:
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75
  1 56 74  1 75 63 60  1 73 64]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the ri




[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1
 56 74  1 75 63 60  1 73 64 71]
                     1
  From fairest creatures we desire increase,
  T

In [76]:
batch_size = 138

buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [79]:
dataset

<BatchDataset shapes: ((138, 130), (138, 130)), types: (tf.int32, tf.int32)>

In [81]:
vocab_size = len(vocab)

embed_dim = 64

rnn_neurons = 1026

In [85]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [83]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)



In [84]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred, from_logits=True)

In [88]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,return_sequences=True, stateful=True, recurrent_initializer = 'glorot_uniform'))
    
    model.add(Dense(vocab_size))
    
    model.compile(optimizer = 'adam',loss= sparse_cat_loss)
    
    return model

In [89]:
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [90]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (138, None, 64)           5376      
_________________________________________________________________
gru_1 (GRU)                  (138, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (138, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [91]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)

In [92]:
example_batch_predictions.shape # (batch_size, seq_len, vocab_len)

TensorShape([138, 130, 84])

In [93]:
example_batch_predictions[0]

<tf.Tensor: id=6604, shape=(130, 84), dtype=float32, numpy=
array([[ 4.0291189e-03,  4.6675741e-03, -1.1014022e-03, ...,
         6.4759329e-03,  3.5999464e-03, -4.8164526e-05],
       [ 4.5796400e-03, -7.0671812e-03, -3.1368337e-03, ...,
         2.3475972e-03,  8.0567150e-04, -2.2737884e-03],
       [ 5.2170567e-03, -8.6278487e-03, -3.9276332e-03, ...,
         9.2355730e-03, -3.7997041e-04, -2.9444760e-03],
       ...,
       [-4.9105857e-04,  1.0426113e-02,  8.9816964e-04, ...,
         1.5491487e-03,  9.5957108e-03,  8.5955504e-03],
       [ 1.5074683e-03, -4.3389313e-03, -1.6279679e-03, ...,
        -2.0568836e-03,  2.8867479e-03,  2.5750170e-03],
       [ 4.1721417e-03,  3.0208007e-03,  1.7854173e-03, ...,
        -4.1132281e-03, -2.1402987e-03, -8.1825051e-03]], dtype=float32)>

In [94]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)

In [96]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [97]:
epochs = 10

In [98]:
model.fit(dataset,epochs=epochs)

In [99]:
#Generating Text